In [1]:
import pandas as pd
import random

In [2]:
#variable to store which student dataset we want to run
student_url = 'data/raw/datasetA_with_roommate_pref.csv'

In [3]:
#variable to store which room dataset we want to run
room_url = 'data/raw/final_room_data.csv'

In [4]:
student_df = pd.read_csv(student_url, index_col= False)
room_df = pd.read_csv(room_url, index_col= False)

In [5]:
#drop unneccessary columns (maybe drop: name, school, is_single_preferred)
student_df = student_df.drop(columns=['date_of_birth', 'is_private_bathroom_preferred', 'laundry_availibility'])
print(student_df.head)

<bound method NDFrame.head of                   name sex       RUID          student_year  \
0     Jennifer Simmons   F  977522906             Sophomore   
1      Courtney Murphy   F  567583671             Sophomore   
2       Alexander King   M  450859644              Freshman   
3      Amanda Atkinson   F  826829351             Sophomore   
4          Mark Rogers   M  852305641                Junior   
...                ...  ..        ...                   ...   
5325  Mr. Joseph White   M  900451724             Doctorate   
5326   Alyssa Franklin   F  933744275             Doctorate   
5327       David James   M  417696262               Masters   
5328     Stephen Heath   M  940035368  Postdoctoral studies   
5329    Daniel Nichols   M  755301395             Doctorate   

                                         student_school  accessibility_need  \
0                                 School of Engineering                   1   
1                           School of Arts and Sciences

# Sorting
We sorted the dataset using the following steps:
1. assigning every student a lottery number & sorting them by it
2. prioritizing freshman students (bringing them to the top of the list regardless of lottery number)
3. prioritizing disability students (bringing them to the top of the list regardless of lottery number)
4. prioritizing low-income students (bringing them to the top of the list regardless of lottery number)

## What this means
All of these categories are sorted by lottery
1. Freshman & Disabled & Low-income
2. Disabled & Low-income
3. Disabled
4. Freshman & Low-income
5. Low-income
6. Freshman
7. Everyone else

In [6]:
# defining variables
total_students = student_df.shape[0]

In [7]:
# definining weights
disability_pts = 20
income_pts = 10
freshman_pts = 5

# assigning weights to each student
weight_df = student_df
weight_df['weights'] = 0
for row in weight_df.index:
    if weight_df.loc[row, 'accessibility_need'] == 1:
        weight_df.at[row, 'weights'] += disability_pts
    if weight_df.loc[row, 'low_income_status'] == 1:
        weight_df.at[row, 'weights'] += income_pts
    if weight_df.loc[row, 'student_year'] == "Freshman":
        weight_df.at[row, 'weights'] += freshman_pts

In [8]:
# will assign a different random number to each student
lottery_df = weight_df
numbers_range = range(1, total_students*4)
lottery_numbers = random.choices(numbers_range, k=total_students)
lottery_df["lottery_number"] = lottery_numbers

In [9]:
# sort student dataset by weight and then lottery
sorted_df = lottery_df.sort_values(by=['weights', 'lottery_number'], ascending=[False, True])
sorted_df.to_csv('data/processed/sorted_students.csv', index=False)

# Matching Algorithm

In [10]:
# setting variables
total_rooms = room_df.shape[0]
graduate_rooms = 0
accessible_rooms = 0

# adding availability columns to dataframe
room_df['available'] = 1

#match to room, based on preference & price?
